![rustronomy_dark_banner](https://github.com/smups/rustronomy/blob/main/logos/Rustronomy-fits_github_banner_dark.png?raw=true#gh-light-mode-only)
# Example 1 - reading a FITS file

Let's start by importing some dependancies:

In [2]:
:dep rustronomy-fits = {git = "https://github.com/smups/rustronomy"}
:dep ndarray = {ndarray = 0.15, features = ["blas", "approx", "std"]}
:dep dirs = {dirs = 4}

In [3]:
use rustronomy_fits as rsf;
use ndarray as nd;
use dirs;

### 1.1 - displaying the structure of a FITS file
When using Rustronomy-fits, it's often important to know what kind of data the FITS file contains _before_ you try to access any of the data contained within. This has to do with Rust being statically typed: you cant just ask for "data contained within HDU0" without knowing the type of the data.

Luckily, we can simply print the structure of a FITS file.

_Note: in this example we are using the example FITS file provided in the example folder on github_

In [4]:
use std::fs;
use std::path;

//We'll be using one of the FITS files from the tests folder
let mut path = dirs::home_dir().unwrap();
path.push("Desktop/code/rustronomy/rustronomy/rustronomy-fits/resources/tests/real_assortment/master_science0.fits");

//open and print the file (we need the mut later on)
let mut fits = rsf::Fits::open(&path)?;
println!("{fits}");


>=================================<|FITS File|>=================================
>Total File size in FITS blocks: 2678
>Number of Header-Data-Units: 2
>Contents:
>-------------------------------------------------------------------------------
>  [HDU #0]
>  Total HDU size in FITS blocks: 1
>    [Header] - size: 1, #records: 4
>    [Data] - (NO_DATA)
>-------------------------------------------------------------------------------
>  [HDU #1]
>  Total HDU size in FITS blocks: 2677
>    [Header] - size: 2, #records: 42
>    [Data] - (IMAGE) datatype: f64, shape: (956,1007), size: 2675
>===============================================================================




from the print-out we can see that we have a `f64` Image in the second header-data-unit. We can get more information about the Image by printing its header:

In [5]:
println!("{}", fits.get_hdu(1).unwrap().get_header());

>================================<|FITS Header|>================================
>Size in FITS blocks: 2
>  [CRVAL1] : 668.00000000 //Reference value on 1st axis in primary WCS
>  [BITPIX] : -64 //
>  [CCDNAME] : A5382-1-7 //Name of detector chip.
>  [CD2_2] : 1.00000000 //Transformation matrix for primary WCS
>  [PCOUNT] : 0 //Number of pixels following data
>  [DASCHAN] : 4 //Number of readout channel
>  [READNOIS] : 5.80000000 //Nominal Readout noise in electrons.
>  [INHERIT] : T //Extension inherits primary HDU.
>  [CCDCHIP] : A5382-1-7 //Name of detector chip.
>  [SATURATE] : 65535.00000000 //Highest value that is unsaturated
>  [PV2_2] : 1.00000000 //Projection coefficient for primary WCS
>  [CCDYPIXE] : 0.00000000 //[m] Size of pixels in y.
>  [GAIN] : 2.90000000 //Nominal Photo-electrons per ADU.
>  [NAXIS2] : 1007 //
>  [NAXIS] : 2 //
>  [WINNO] : 4 //Number of readout window
>  [CRVAL2] : 1642.00000000 //Reference value on 2nd axis in primary WCS
>  [EXTNAME] : extension1 //

### 1.2 - loading the data as a ndarray

In general, the `get_xyz()` methods in rustronomy-fits return references (unless the return type is primitive). To obtain an owned version of the data and header contained within the hdu, we have to use a `remove_hdu()` method to obtain an owned version of the hdu, which we then deconstruct using the `to_parts()` method:

In [6]:
let (header, data) = fits.remove_hdu(1).unwrap().to_parts();
println!("{fits}"); //note that hdu #1 is missing!


>=================================<|FITS File|>=================================
>Total File size in FITS blocks: 1
>Number of Header-Data-Units: 1
>Contents:
>-------------------------------------------------------------------------------
>  [HDU #0]
>  Total HDU size in FITS blocks: 1
>    [Header] - size: 1, #records: 4
>    [Data] - (NO_DATA)
>===============================================================================




Our next task is to convert the data that we received to something usefull. FITS files can contain various data containers, such as Images and Tables. These are represented in rsf with the `Extension` enum. `data` is one of the variants of this extension enum. In our case, we know that we have a FITS Image, which is represented as the `Image` variant. 

The actual image is contained within the enum variant. We can turn it into a ndarray by calling the `as_owned_..._array` method where we replace ... with the datatype of the image, in our case: f64:

In [7]:
let array = match data.unwrap() {
    rsf::Extension::Image(img) => img.as_owned_f64_array()?,
    _ => panic!()
};
println!("{array}");

[[1422.7801659179809, 1399.816122322237, 1338.7904151430416, 1408.4804134009175, 1366.8482227571476, ..., 1358.3960997166196, 1352.7671125409004, 1340.294613536379, 1349.777574789123, 1398.2811824258586],
 [1378.4421510448844, 1360.8390147613043, 1406.6388090478217, 1361.0390411373935, 1358.2108106679993, ..., 1390.0750628492162, 1339.3917192221113, 1344.7475183299823, 1425.7183909398113, 1338.4564511564847],
 [1373.1366001971037, 1370.5883535269681, 1338.0797209894872, 1402.467127919193, 1368.0761952872153, ..., 1378.722279552521, 1346.6757815067479, 1376.1458382143492, 1367.7188887359011, 1396.748142561798],
 [1378.656539983296, 1361.3283115425008, 1384.0243696509249, 1323.9161909886272, 1352.965946696271, ..., 1343.4366834421842, 1373.372527628058, 1368.435232332786, 1409.1756516805374, 1358.8479562777227],
 [1312.5626618547906, 1325.8930440683264, 1401.9297771777542, 1427.8927905490139, 1368.9676818720338, ..., 1347.4136319295308, 1383.814735960337, 1418.686204058713, 1359.05959299